In [ ]:
import speedup
import tensorflow as tf
import numpy as np
import tensorflow as tf
import random
random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)

In [ ]:
def double_conv_block(x, n_filters):
   x = tf.keras.layers.Conv2D(n_filters, (3, 3), activation = "relu", kernel_initializer = "he_normal", padding="same")(x)
   x = tf.keras.layers.BatchNormalization()(x)
   x = tf.keras.layers.Conv2D(n_filters, (3, 3), activation = "relu", kernel_initializer = "he_normal", padding="same")(x)
   x = tf.keras.layers.BatchNormalization()(x)
   return x

In [ ]:
def downsample_block(x, n_filters):
   f = double_conv_block(x, n_filters)
   p = tf.keras.layers.MaxPool2D((2, 2))(f)
   return f, p

In [ ]:
def upsample_block(x, conv_features, n_filters):
   x = tf.keras.layers.Conv2DTranspose(n_filters, (3, 3), (2, 2), padding="same")(x)
   x = tf.keras.layers.concatenate([x, conv_features])
   x = double_conv_block(x, n_filters)
   return x

In [ ]:
class QuadLayer(tf.keras.layers.Layer):
  def __init__(self):
    super(QuadLayer, self).__init__()

  def build(self, input_shape):
    self.kernel = self.add_weight("kernel",
                                  shape=[6, 3])

  def call(self, o, m):
    result1 = self.kernel[0, 0] * o[:, :, :, 0] + self.kernel[1, 0] * o[:, :, :, 1] + self.kernel[2, 0] * o[:, :, :, 2] + \
      self.kernel[3, 0] * o[:, :, :, 0] * m[:, :, :, 0] + self.kernel[4, 0] * o[:, :, :, 1] * m[:, :, :, 1] + self.kernel[5, 0] * o[:, :, :, 2] * m[:, :, :, 2]
    result2 = self.kernel[0, 1] * o[:, :, :, 0] + self.kernel[1, 1] * o[:, :, :, 1] + self.kernel[2, 1] * o[:, :, :, 2] + \
      self.kernel[3, 1] * o[:, :, :, 0] * m[:, :, :, 0] + self.kernel[4, 1] * o[:, :, :, 1] * m[:, :, :, 1] + self.kernel[5, 1] * o[:, :, :, 2] * m[:, :, :, 2]
    result3 = self.kernel[0, 2] * o[:, :, :, 0] + self.kernel[1, 2] * o[:, :, :, 1] + self.kernel[2, 2] * o[:, :, :, 2] + \
      self.kernel[3, 2] * o[:, :, :, 0] * m[:, :, :, 0] + self.kernel[4, 2] * o[:, :, :, 1] * m[:, :, :, 1] + self.kernel[5, 2] * o[:, :, :, 2] * m[:, :, :, 2]
    return tf.stack([result1, result2, result3], 3)

In [ ]:
quad_layer = QuadLayer()

In [ ]:
imageSize = 512
m = 3

def constructModel():
    inputs = tf.keras.Input(shape=(imageSize, imageSize, m))
    f1, p1 = downsample_block(inputs, 64)
    f2, p2 = downsample_block(p1, 128)
    f3, p3 = downsample_block(p2, 256)
    f4, p4 = downsample_block(p3, 512)
    

    bottleneck = double_conv_block(p4, 1024)

    u6 = upsample_block(bottleneck, f4, 512)
    u7 = upsample_block(u6, f3, 256)
    u8 = upsample_block(u7, f2, 128)
    u9 = upsample_block(u8, f1, 64)
    pre_outputs = tf.keras.layers.Conv2D(3, (1, 1), activation='sigmoid', padding = "same")(u9)

    outputs = quad_layer(inputs, pre_outputs)

    return tf.keras.Model(inputs, outputs, name="U-Net")

In [ ]:
model_instance = constructModel()
model_instance.summary()

Model: "U-Net"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_171 (Conv2D)            (None, 512, 512, 64  1792        ['input_10[0][0]']               
                                )                                                                 
                                                                                                  
 batch_normalization_162 (Batch  (None, 512, 512, 64  256        ['conv2d_171[0][0]']             
 Normalization)                 )                                                             

In [ ]:
image_path = '/content/drive/MyDrive/source2'
models_path = '/content/drive/MyDrive/models/'

In [ ]:
import itertools
import tensorflow as tf
from speedup import generate_out_images3
import numpy as np
from random import randint, uniform
import imageio
import time


source_num = 2799
dim = 512
stationary_defocus = 0.05


def gen():
    while True:
        layer1_number = randint(0, source_num)
        layer2_number = randint(0, source_num)
        layer3_number = randint(0, source_num)

        src1 = imageio.imread(image_path + '/image' + str(layer1_number).zfill(4) + '.png')
        src2 = imageio.imread(image_path + '/image' + str(layer2_number).zfill(4) + '.png')
        src3 = imageio.imread(image_path + '/image' + str(layer3_number).zfill(4) + '.png')
        src = np.zeros((dim, dim, m), np.double)
        src[:, :, 0] = src1[:, :, 0]
        src[:, :, 1] = src2[:, :, 0]
        src[:, :, 2] = src3[:, :, 0]
        src = src - np.amin(src)
        src = src / np.amax(src)

        w = uniform(0.05, 0.5) 
        
        a_10 = uniform(-1e3, 1e3)
        a_01 = uniform(-1e3, 1e3)
        b_20 = uniform(1, 1.5)
        b_11 = uniform(-0.1, 0.1)
        b_02 = uniform(1, 1.5)
        c_30 = uniform(-1.5e-6, 1.5e-6)
        c_21 = uniform(-2e-6, 2e-6)
        c_12 = uniform(-2e-6, 2e-6)
        c_03 = uniform(-1.5e-6, 1.5e-6)

        out = generate_out_images3(dim, m, w, stationary_defocus, a_10, a_01, b_20, b_11, b_02, c_30, c_21, c_12, c_03, src)[1]

        out = out / np.amax(out)

        yield (out, src)


tr_dataset = tf.data.Dataset.from_generator(
     gen, (tf.float64, tf.float64), (tf.TensorShape([dim, dim, m]), tf.TensorShape([dim, dim, m])))\
    .batch(batch_size=2).prefetch(buffer_size=8)

val_dataset = tf.data.Dataset.from_generator(
     gen, (tf.float64, tf.float64), (tf.TensorShape([dim, dim, m]), tf.TensorShape([dim, dim, m])))\
    .take(count=128).cache().batch(batch_size=2)


opt = tf.keras.optimizers.Adam()

save_best_callback = tf.keras.callbacks.ModelCheckpoint(models_path + 'bestmodel_unet_conv2d_custom_layer.hdf5',
                                                        save_weights_only=True,save_best_only=True, verbose=True)
csv_logger_callback = tf.keras.callbacks.CSVLogger(models_path + 'log_unet_conv2d_custom_layer.csv')
lr_reduce_callback = tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, min_delta=5e-4, patience=5)
early_stop_callback = tf.keras.callbacks.EarlyStopping(patience=25)

model_instance = constructModel()
model_instance.compile(loss='mse', optimizer=opt, metrics=['binary_crossentropy', 'mse'])
model_instance.fit(x=tr_dataset, validation_data=val_dataset, verbose=1, validation_steps=64,
                   steps_per_epoch=256, epochs=200,
                   callbacks=[save_best_callback, csv_logger_callback, lr_reduce_callback, early_stop_callback])